# Utilização de SQLalchemy

Para inicialização e manipulação de um banco de dados criado com SQLite

## Setup

Acesso ao banco de dados e criação do objeto `session` para acesso a seção SQL

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

engine = create_engine("sqlite:///Alunos15.db", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

## Migration

População do banco de dados com tabelas

In [2]:
import enum
from sqlalchemy import (
    String,
    Enum,
    PrimaryKeyConstraint as pkc,
    UniqueConstraint as unique,
    CheckConstraint as constraint,
    ForeignKey as fk,
    func,
    select
)
from sqlalchemy.orm import (
    Mapped,
    Session,
    relationship,
    column_property,
    mapped_column as column,
    validates
)
from typing import Any, Optional
from utils import (
    Base,
    BaseModel,
    defaultPrimaryKey as pk,
    parentPrimaryKey as ppk,
    foreignKeyCascade as fkc,
    backref,
    childOf,
    positive
)

Base.metadata.drop_all(engine)
Base.metadata.clear()

class Person(BaseModel):
    id: Mapped[str] = column(String(14), primary_key=True)
    name: Mapped[str] = column(String(40))
    age: Mapped[int | None] = positive("age")
    city: Mapped[str | None] = column(String(30))

    # Relationships
    student: Mapped[Optional["Student"]] = childOf("person")
    professor: Mapped[Optional["Professor"]] = childOf("person")
    

class Professor(BaseModel):
    id: Mapped[str] = ppk("person")
    func_id: Mapped[str] = column(String(7), unique=True)
    level: Mapped[str | None] = column(String(7))
    field_of_study: Mapped[str | None] = column(String(20))

    # Relationships
    person: Mapped["Person"] = backref("professor")
    classes: Mapped[list["Class"]] = relationship(
        secondary="teaches",
        back_populates="professor"
    )


class Student(BaseModel):
    id: Mapped[str] = ppk("person")
    nUSP: Mapped[int] = column(autoincrement=True, unique=True)
    course: Mapped[str | None] = column(String(15))

    # Relationships
    person: Mapped["Person"] = backref("student")
    classes: Mapped[list["Class"]] = relationship(
        secondary="enrollment",
        back_populates="students"
    )


class Enrollment(BaseModel):
    student_nUSP: Mapped[int] = fkc("student.nUSP")
    class_id: Mapped[str] = fkc("class.id")
    grade: Mapped[float | None] = column(
        constraint('grade >= 0 AND grade <= 10')
    )

    __table_args__: tuple[pkc,] = (
        pkc("student_nUSP", "class_id"),
    )


class Class(BaseModel):
    id: Mapped[int] = pk()
    subject_id: Mapped[str] = fkc("subject.id")
    year: Mapped[int | None] = positive("year")

    # Dynamic attribute to count students enrolled in this class
    student_count: Mapped[int] = column_property(
        select(func.count(Enrollment.student_nUSP))
        .where(Enrollment.class_id == id)
        .scalar_subquery(),
        deferred=True
    )

    # Relationships
    subject: Mapped["Subject"] = backref("classes")
    students: Mapped[list["Student"]] = relationship(
        secondary="enrollment",
        back_populates="classes"
    )
    professor: Mapped["Professor"] = relationship(
        secondary="teaches",
        back_populates="classes"
    )
    schedule: Mapped[list["Schedule"]] = childOf("class_rel")


class Weekday(enum.Enum):
    MONDAY = "Monday"
    TUESDAY = "Tuesday"
    WEDNESDAY = "Wednesday"
    THURSDAY = "Thursday"
    FRIDAY = "Friday"
    SATURDAY = "Saturday"

Timeslot = enum.Enum(
    "Timeslot",
    [(slot, i) for i, slot in enumerate([
        "8:10 – 9:40",
        "10:00 – 11:30",
        "13:00 – 14:30",
        "14:50 – 16:20",
        "19:00 – 20:30",
        "20:40 – 22:10",
    ])]
)

class Schedule(BaseModel):
    timeslot: Mapped[str] = column(Enum(Timeslot))
    weekday: Mapped[str] = column(Enum(Weekday))

    # Foreign keys
    class_id: Mapped[str] = column(fk("class.id"))

    # Relationships
    class_rel: Mapped["Class"] = backref("schedule")

    __table_args__: tuple[pkc,] = (
        pkc("timeslot", "weekday", "class_id"),
    )    


class Subject(BaseModel):
    id: Mapped[str] = column(String(7), primary_key=True)
    name: Mapped[str] = column(String(25))
    credits: Mapped[int] = positive("credits")

    # Relationships
    prerequisites: Mapped[list["Subject"]] = relationship(
        secondary="subject_prerequisite",
        primaryjoin="SubjectPrerequisite.subject_id == Subject.id",
        secondaryjoin="SubjectPrerequisite.prerequisite_id == Subject.id",
        back_populates="required_by",
    )
    required_by: Mapped[list["Subject"]] = relationship(
        secondary="subject_prerequisite",
        primaryjoin="SubjectPrerequisite.prerequisite_id == Subject.id",
        secondaryjoin="SubjectPrerequisite.subject_id == Subject.id",
        back_populates="prerequisites",
    )
    classes: Mapped[list["Class"]] = childOf("subject")


class SubjectPrerequisite(BaseModel):
    subject_id: Mapped[str] = ppk("subject")
    prerequisite_id: Mapped[str] = column(fk("subject.id"))

    # Validate if prerequisite does not entail a circular dependency.
    @validates('prerequisite')
    def validate_prerequisite(self, key: str, prerequisite_id: str) -> str:

        # Get the current database session
        session = Session.object_session(self)
        if not session:
            raise RuntimeError("No database session available for validation")

        visited = set()
        stack = [prerequisite]
        while stack:
            requirement_id = stack.pop()
            if requirement_id in visited:
                continue
            if requirement_id == self.subject_id:
                raise ValueError("Circular dependency detected: subject cannot require itself")
            visited.add(requirement_id)
            # Fetch all prerequisites of requirement_id
            requirements = session.query(SubjectPrerequisite.prerequisite_id).filter(
                SubjectPrerequisite.subject_id == requirement_id
            ).scalars().all()
            stack.extend(requirements)
        return prerequisite
        

class Teaches(BaseModel):
    func_id: Mapped[str] = fkc("professor.func_id")
    class_id: Mapped[str] = fkc("class.id")
    textbook: Mapped[str | None] = column(String(50))

    __table_args__: tuple[pkc, ] = (
        pkc("func_id", "class_id"),
    )    


Base.metadata.create_all(engine)

2025-08-14 15:59:40,090 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-14 15:59:40,091 INFO sqlalchemy.engine.Engine COMMIT
2025-08-14 15:59:40,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-14 15:59:40,131 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("person")
2025-08-14 15:59:40,132 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-14 15:59:40,134 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("professor")
2025-08-14 15:59:40,136 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-14 15:59:40,137 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("student")
2025-08-14 15:59:40,138 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-14 15:59:40,139 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("enrollment")
2025-08-14 15:59:40,140 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-14 15:59:40,141 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("class")
2025-08-14 15:59:40,143 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-14 15:59:40,

## Inserção de registros

In [3]:
# Adicionar professores
session.add(Person(id="11111111111111", name="Ari", age=25, city='Rio Claro'))
session.add(Person(id="11111111111112", name="Adão", age=30, city='Ibitiga'))
session.add(Person(id="11111111111113", name="Alselmo", age=31, city='São Carlos'))
session.add(Person(id="11111111111114", name="Amalia", age=39))
session.add(Person(id="11111111111115", name="Ana", age=31, city='São Paulo'))
session.add(Person(id="11111111111116", name="Alice", age=35, city='Araras'))
session.add(Person(id="11111111111117", name="Amauri", age=34, city='São Carlos'))
session.add(Person(id="11111111111118", name="Artur", age=41, city='São Carlos'))
session.add(Person(id="11111111111119", name="Adriana", age=45, city='Araraquara'))

session.add(Professor(id="11111111111111", func_id="1111", level='MS-1', field_of_study='Matemática'))
session.add(Professor(id="11111111111112", func_id="2222", level='MS-2', field_of_study='Computação'))
session.add(Professor(id="11111111111113", func_id="3333", level='MS-2', field_of_study='Computação'))
session.add(Professor(id="11111111111114", func_id="8888", level='MS-3'))
session.add(Professor(id="11111111111115", func_id="4444", level='MS-3', field_of_study='Elétrica'))
session.add(Professor(id="11111111111116", func_id="5555", level='MS-3', field_of_study='Física'))
session.add(Professor(id="11111111111117", func_id="6666", level='MS-3', field_of_study='Elétrica'))
session.add(Professor(id="11111111111118", func_id="7777", level='MS-4', field_of_study='Computação'))
session.add(Professor(id="11111111111119", func_id="9999", level='MS-5', field_of_study='Computação'))

# Adicionar alunos
session.add(Person(id="11111111111120", name="Carlos", age=21, city="Sao Carlos"))
session.add(Person(id="11111111111121", name="Celso", age=22, city="Sao Carlos"))
session.add(Person(id="11111111111122", name="Cicero", age=22, city="Araraquara"))
session.add(Person(id="11111111111123", name="Carlitos", age=21, city="Ibitinga"))
session.add(Person(id="11111111111124", name="Catarina", age=23, city="Sao Carlos"))
session.add(Person(id="11111111111125", name="Cibele", age=21, city="Araraquara"))
session.add(Person(id="11111111111126", name="Corina", age=25, city="Rio Claro"))
session.add(Person(id="11111111111127", name="Celina", age=27, city="Sao Carlos"))
session.add(Person(id="11111111111128", name="Celia", age=20, city="Rio Claro"))
session.add(Person(id="11111111111129", name="Cesar", age=21, city="Araraquara"))
session.add(Person(id="11111111111130", name="Denise", age=35, city="Ibate"))
session.add(Person(id="11111111111131", name="Durval"))
session.add(Person(id="11111111111132", name="Daniel", age=19, city="Campinas"))
session.add(Person(id="11111111111133", name="Dora", age=24))
session.add(Person(id="11111111111134", name="Dina", city="Campinas"))

session.add(Student(id="11111111111120", nUSP=1234, course="Computação"))
session.add(Student(id="11111111111121", nUSP=2345, course="Computação"))
session.add(Student(id="11111111111122", nUSP=3456, course="Matemática"))
session.add(Student(id="11111111111123", nUSP=4567, course="Computação"))
session.add(Student(id="11111111111124", nUSP=5678, course="Elétrica"))
session.add(Student(id="11111111111125", nUSP=6789, course="Computação"))
session.add(Student(id="11111111111126", nUSP=7890, course="Matemática"))
session.add(Student(id="11111111111127", nUSP=8901, course="Computação"))
session.add(Student(id="11111111111128", nUSP=9012, course="Computação"))
session.add(Student(id="11111111111129", nUSP=9123, course="Elétrica"))
session.add(Student(id="11111111111130", nUSP=4584, course="Matemática"))
session.add(Student(id="11111111111131", nUSP=1479, course="Computação"))
session.add(Student(id="11111111111132", nUSP=1489, course="Computação"))
session.add(Student(id="11111111111133", nUSP=1469, course="Geografia"))
session.add(Student(id="11111111111134", nUSP=1459))

# Adicionar disciplinas
session.add(Subject(id='SCE-179', name='Base de Dados', credits=4))
session.add(Subject(id='SMA-179', name='Algebra', credits=3))
session.add(Subject(id='SCE-200', name='Lab. Base de Dados', credits=4))
session.add(SubjectPrerequisite(subject_id='SCE-179', prerequisite_id='SMA-179'))
session.add(SubjectPrerequisite(subject_id='SCE-200', prerequisite_id='SMA-179'))

# Adicionar turmas
session.add(Class(subject_id='SCE-179', year=2024))
session.add(Class(subject_id='SMA-179', year=2023))
session.add(Class(subject_id='SMA-179', year=2024))
session.add(Class(subject_id='SCE-200', year=2023))
session.add(Class(subject_id='SCE-200', year=2024))
session.add(Class(subject_id='SCE-200', year=2024))

# Adicionar horários
session.add(Schedule(class_id=1, timeslot=Timeslot["8:10 – 9:40"], weekday=Weekday.MONDAY))
session.add(Schedule(class_id=1, timeslot=Timeslot["8:10 – 9:40"], weekday=Weekday.WEDNESDAY))
session.add(Schedule(class_id=2, timeslot=Timeslot["10:00 – 11:30"], weekday=Weekday.MONDAY))
session.add(Schedule(class_id=2, timeslot=Timeslot["8:10 – 9:40"], weekday=Weekday.WEDNESDAY))
session.add(Schedule(class_id=2, timeslot=Timeslot["14:50 – 16:20"], weekday=Weekday.FRIDAY))
session.add(Schedule(class_id=3, timeslot=Timeslot["10:00 – 11:30"], weekday=Weekday.MONDAY))
session.add(Schedule(class_id=3, timeslot=Timeslot["10:00 – 11:30"], weekday=Weekday.WEDNESDAY))
session.add(Schedule(class_id=3, timeslot=Timeslot["19:00 – 20:30"], weekday=Weekday.FRIDAY))
session.add(Schedule(class_id=4, timeslot=Timeslot["8:10 – 9:40"], weekday=Weekday.TUESDAY))
session.add(Schedule(class_id=5, timeslot=Timeslot["10:00 – 11:30"], weekday=Weekday.TUESDAY))
session.add(Schedule(class_id=6, timeslot=Timeslot["13:00 – 14:30"], weekday=Weekday.TUESDAY))

# Adicionar matrículas
session.add(Enrollment(student_nUSP=1234, class_id=1, grade=8.0))    # 100 → 1
session.add(Enrollment(student_nUSP=2345, class_id=1, grade=9.0))    # 100 → 1
session.add(Enrollment(student_nUSP=4567, class_id=1, grade=7.0))    # 100 → 1
session.add(Enrollment(student_nUSP=8901, class_id=1, grade=4.0))    # 100 → 1
session.add(Enrollment(student_nUSP=9123, class_id=1, grade=9.0))    # 100 → 1
session.add(Enrollment(student_nUSP=3456, class_id=1, grade=7.0))    # 100 → 1
session.add(Enrollment(student_nUSP=9012, class_id=1, grade=6.0))    # 100 → 1

session.add(Enrollment(student_nUSP=2345, class_id=2, grade=4.0))    # 101 → 2
session.add(Enrollment(student_nUSP=3456, class_id=2, grade=2.0))    # 101 → 2
session.add(Enrollment(student_nUSP=2344, class_id=2, grade=3.0))    # 101 → 2
session.add(Enrollment(student_nUSP=4567, class_id=2, grade=4.0))    # 101 → 2
session.add(Enrollment(student_nUSP=6789, class_id=2, grade=6.0))    # 101 → 2
session.add(Enrollment(student_nUSP=7890, class_id=2, grade=10.0))   # 101 → 2
session.add(Enrollment(student_nUSP=8901, class_id=2, grade=3.0))    # 101 → 2
session.add(Enrollment(student_nUSP=9012, class_id=2, grade=9.0))    # 101 → 2
session.add(Enrollment(student_nUSP=1234, class_id=2, grade=9.0))    # 101 → 2

session.add(Enrollment(student_nUSP=2345, class_id=3, grade=7.0))    # 102 → 3
session.add(Enrollment(student_nUSP=3456, class_id=3, grade=9.0))    # 102 → 3
session.add(Enrollment(student_nUSP=4567, class_id=3, grade=10.0))   # 102 → 3
session.add(Enrollment(student_nUSP=5678, class_id=3, grade=7.0))    # 102 → 3
session.add(Enrollment(student_nUSP=9123, class_id=3, grade=9.0))    # 102 → 3
session.add(Enrollment(student_nUSP=8901, class_id=3, grade=9.0))    # 102 → 3

session.add(Enrollment(student_nUSP=2345, class_id=4, grade=7.0))    # 104 → 4
session.add(Enrollment(student_nUSP=3456, class_id=4, grade=10.0))   # 104 → 4
session.add(Enrollment(student_nUSP=4567, class_id=4, grade=4.0))    # 104 → 4
session.add(Enrollment(student_nUSP=6789, class_id=4, grade=5.0))    # 104 → 4
session.add(Enrollment(student_nUSP=7890, class_id=4, grade=9.0))    # 104 → 4
session.add(Enrollment(student_nUSP=8901, class_id=4, grade=8.0))    # 104 → 4
session.add(Enrollment(student_nUSP=9012, class_id=4, grade=9.0))    # 104 → 4
session.add(Enrollment(student_nUSP=1234, class_id=4, grade=4.0))    # 104 → 4
session.add(Enrollment(student_nUSP=5678, class_id=4, grade=8.0))    # 104 → 4
session.add(Enrollment(student_nUSP=9123, class_id=4, grade=7.0))    # 104 → 4

session.add(Enrollment(student_nUSP=4567, class_id=5, grade=7.0))    # 105 → 5
session.add(Enrollment(student_nUSP=1459, class_id=5, grade=0.0))    # 105 → 5
session.add(Enrollment(student_nUSP=1469, class_id=5, grade=0.0))    # 105 → 5
session.add(Enrollment(student_nUSP=1479, class_id=5))             # 105 → 5

# Relaciona professores com turmas
session.add(Teaches(func_id="1111", class_id=1, textbook="Database Intro"))            # 100 → 1
session.add(Teaches(func_id="1111", class_id=4, textbook="Bases de Dados na Pratica")) # 103 → 4
session.add(Teaches(func_id="2222", class_id=2, textbook="Algebra para Todos"))       # 101 → 2
session.add(Teaches(func_id="3333", class_id=1, textbook="Database Intro"))            # 100 → 1
session.add(Teaches(func_id="3333", class_id=4, textbook="Bases de Dados na Pratica")) # 103 → 4
session.add(Teaches(func_id="3333", class_id=5, textbook="Bases de Dados na Pratica")) # 104 → 5
session.add(Teaches(func_id="3333", class_id=3, textbook="Algebra Moderna"))           # 102 → 3
session.add(Teaches(func_id="4444", class_id=1, textbook="Database Intro"))            # 100 → 1
session.add(Teaches(func_id="4444", class_id=6, textbook="Bases de Dados"))            # 105 → 6
session.add(Teaches(func_id="5555", class_id=2, textbook="Algebra para Todos"))        # 101 → 2
session.add(Teaches(func_id="5555", class_id=3, textbook="Algebra Moderna"))           # 102 → 3
session.add(Teaches(func_id="6666", class_id=5, textbook="Introducao a bases de dados")) # 104 → 5
session.add(Teaches(func_id="7777", class_id=2, textbook="Algebra Moderna"))           # 101 → 2
session.add(Teaches(func_id="7777", class_id=3, textbook="Algebra para Todos"))        # 102 → 3
session.add(Teaches(func_id="9999", class_id=1, textbook="Database Intro"))            # 100 → 1

session.commit()

2025-08-14 15:59:41,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-14 15:59:41,679 INFO sqlalchemy.engine.Engine INSERT INTO enrollment ("student_nUSP", class_id, grade) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), ... 161 characters truncated ...  ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING created, updated, "student_nUSP", class_id
2025-08-14 15:59:41,680 INFO sqlalchemy.engine.Engine [generated in 0.00022s (insertmanyvalues) 1/1 (ordered)] (1234, 1, 8.0, 2345, 1, 9.0, 4567, 1, 7.0, 8901, 1, 4.0, 9123, 1, 9.0, 3456, 1, 7.0, 9012, 1, 6.0, 2345, 2, 4.0, 3456, 2, 2.0, 2344, 2, 3.0, 4567, 2, 4.0, 6789, 2, 6.0, 7890, 2, 10.0, 8901, 2, 3.0, 9012, 2, 9.0, 1234, 2, 9.0, 2345, 3 ... 8 parameters truncated ... 3, 7.0, 9123, 3, 9.0, 8901, 3, 9.0, 2345, 4, 7.0, 3456, 4, 10.0, 4567, 4, 4.0, 6789, 4, 5.0, 7890, 4, 9.0, 8901, 4

In [ ]:
session.rollback()